In [1]:
!pip install pulp
import pandas as pd
from pulp import *

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 18.9 MB/s eta 0:00:00


In [ ]:
manvar_costs = pd.read_excel('variable_costs.xlsx', index_col = 0)
print("VARIABLE COST FOR EACH PLANT")
manvar_costs

VARIABLE COST FOR EACH PLANT


,USA,Germany,Japan,Brazil,India
Variable Costs ($/Unit),,,,,
USA,12,12,12,12,12
Germany,13,13,13,13,13
Japan,10,10,10,10,10
Brazil,8,8,8,8,8
India,5,5,5,5,5


In [ ]:
# Importing Costs
freight_costs = pd.read_excel('freight_costs.xlsx', index_col = 0)
print("FIXED COST FOR EACH PLANT")
freight_costs

FIXED COST FOR EACH PLANT


,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
USA,0,12250,1100,16100,8778
Germany,13335,0,8617,20244,10073
Japan,15400,22750,0,43610,14350
Brazil,16450,22050,28000,0,29750
India,13650,15400,24500,29400,0


In [ ]:
# Variable Costs
total_var_cost = freight_costs/1000 + manvar_costs
print("TOTAL VARIABLE COST FOR EACH PLANT")
total_var_cost

TOTAL VARIABLE COST FOR EACH PLANT


,USA,Germany,Japan,Brazil,India
Freight Costs ($/Container),,,,,
USA,12.000,24.25,13.100,28.100,20.778
Germany,26.335,13.00,21.617,33.244,23.073
Japan,25.400,32.75,10.000,53.610,24.350
Brazil,24.450,30.05,36.000,8.000,37.750
India,18.650,20.40,29.500,34.400,5.000


In [ ]:
# Importing Costs
fixed_costs = pd.read_excel('fixed_cost.xlsx', index_col = 0)
print("FIXED COST FOR EACH PLANT")
fixed_costs

FIXED COST FOR EACH PLANT


,Low,High
USA,6500,9500
Germany,4980,7270
Japan,6230,9100
Brazil,3230,4730
India,2110,6160


In [ ]:
# Two types of plants: Low Capacity , High Capacity Plant
print("CAPACITY FOR EACH PLANT")

capacity = pd.read_excel('capacity.xlsx', index_col = 0)
capacity

CAPACITY FOR EACH PLANT


,Low,High
Capacity (kUnits/month),,
USA,500,1500
Germany,500,1500
Japan,500,1500
Brazil,500,1500
India,500,3000


In [ ]:
# Demand
print("DEMAND FOR EACH PLANT")
demand = pd.read_excel('demand.xlsx', index_col = 0)
demand


DEMAND FOR EACH PLANT


,Demand
(Units/month),
USA,2800000
Germany,90000
Japan,1700000
Brazil,145000
India,160000


In [ ]:
# Define Decision Variables
loc = ['USA', 'Germany', 'Japan', 'Brazil', 'India']
size = ['Low', 'High']

# Initialize Class
model = LpProblem("Capacitated Plant Location Model", LpMinimize)


# Create Decision Variables
x = LpVariable.dicts("production_", [(i,j) for i in loc for j in loc],
                     lowBound=0, upBound=None, cat='continuous')
y = LpVariable.dicts("plant_",
                     [(i,s) for s in size for i in loc], cat='Binary')

print('Prinitng x and y')
print(x)
print(y)

# Define Objective Function
model += (lpSum([fixed_costs.loc[i,s] * y[(i,s)] * 1000 for s in size for i in loc])
          + lpSum([total_var_cost.loc[i,j] * x[(i,j)]   for i in loc for j in loc]))

# Add Constraints
for location_j in loc:
    model += lpSum([x[(i, location_j)] for i in loc]) == demand.loc[location_j, 'Demand']

for location_i in loc:
    model += lpSum([x[(location_i, j)] for j in loc]) <= lpSum([cap.loc[location_i, s] * y[(location_i, s)] * 1000
                                                                for s in size])


# Define logical constraint: Add a logical constraint so that if the high capacity plant in USA is open, then a low capacity plant in Germany is also opened.
# model += y[('USA','High_Cap')] <= y[('Germany','Low_Cap')]

# Solve Model
model.solve()
print("Total Costs = {:,} ($/Month)".format(int(value(model.objective))))
print('\n' + "Status: {}".format(LpStatus[model.status]))


# Dictionnary
dict_plant = {}
dict_prod = {}
for variable in model.variables():
    if 'plant' in variable.name:
        name = variable.name.replace('plant__', '').replace('_', '')
        dict_plant[name] = int(variable.varValue)
        p_name = name
    else:
        name = variable.name.replace('production__', '').replace('_', '')
        dict_prod[name] = variable.varValue
    print(name, "=", variable.varValue)



Prinitng x and y
{('USA', 'USA'): production__('USA',_'USA'), ('USA', 'Germany'): production__('USA',_'Germany'), ('USA', 'Japan'): production__('USA',_'Japan'), ('USA', 'Brazil'): production__('USA',_'Brazil'), ('USA', 'India'): production__('USA',_'India'), ('Germany', 'USA'): production__('Germany',_'USA'), ('Germany', 'Germany'): production__('Germany',_'Germany'), ('Germany', 'Japan'): production__('Germany',_'Japan'), ('Germany', 'Brazil'): production__('Germany',_'Brazil'), ('Germany', 'India'): production__('Germany',_'India'), ('Japan', 'USA'): production__('Japan',_'USA'), ('Japan', 'Germany'): production__('Japan',_'Germany'), ('Japan', 'Japan'): production__('Japan',_'Japan'), ('Japan', 'Brazil'): production__('Japan',_'Brazil'), ('Japan', 'India'): production__('Japan',_'India'), ('Brazil', 'USA'): production__('Brazil',_'USA'), ('Brazil', 'Germany'): production__('Brazil',_'Germany'), ('Brazil', 'Japan'): production__('Brazil',_'Japan'), ('Brazil', 'Brazil'): production__

/usr/local/lib/python3.10/dist-packages/pulp/pulp.py:1316: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


In [ ]:
# Capacity Plant
list_low, list_high = [], []
for l in loc:
    for capacity in ['Low', 'High']:
        x = "('{}','{}')".format(l, capacity)
        list_low.append(dict_plant[x]) if capacity == 'Low' else list_high.append(dict_plant[x])

df_capacity = pd.DataFrame({'Location': loc, 'Low': list_low, 'High': list_high}).set_index('Location')

df_capacity

,Low,High
Location,,
USA,0,1
Germany,0,0
Japan,0,1
Brazil,1,0
India,0,1
